In [1]:
import pandas as pd
import numpy as np
import matplotlib
import tensorflow as tf
import torch
learning_rate = 0.005
max_iter = 1500
batch_size=64
block_size=32
eval_iters=50
eval_interval=250
n_embd=192
n_head=2
n_layer=4
dropout=0.2

In [2]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device) #So I do not have a GPU so we use the CPU itself for going forward

cpu


In [3]:
file_path = "C:\\Users\\adity\\Documents\\LLM_training.txt"

try:
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    print(text)
except PermissionError as e:
    print(f"PermissionError: {e}")


﻿The Project Gutenberg eBook of Common Diseases of Farm Animals
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Common Diseases of Farm Animals

Author: Robert Alexander Craig

Release date: July 1, 2005 [eBook #8502]
                Most recently updated: January 25, 2013

Language: English

Credits: Produced by Charles Aldarondo, Tiffany Vergon, Beth Trapaga, Charles Franks and the Online Distributed Proofreading Team


*** START OF THE PROJECT GUTENBERG EBOOK COMMON DISEASES OF FARM ANIMALS ***




Produced by Charles Aldarondo, Tiffany Vergon, Beth Trapaga,
Charles Franks and the O

In [4]:
chars=sorted(set(text))
print(chars)
vocab_size=len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']
92


In [5]:
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode= lambda s:[string_to_int[i] for i in s]
decode=lambda l: ''.join([int_to_string[i] for i in l])
#Here we are basically just trying to create a dictionary and encoding the characters with the help of its keys, this is a character level tokeniser

In [6]:
print(encode("Hello"))

[35, 62, 69, 69, 72]


In [7]:
data=torch.tensor(encode(text), dtype=torch.long)
print(data[:50]) #We have encoded our text given above

tensor([91, 47, 65, 62,  1, 43, 75, 72, 67, 62, 60, 77,  1, 34, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 29, 72, 72, 68,  1, 72, 63,  1, 30, 72, 70, 70,
        72, 71,  1, 31, 66, 76, 62, 58, 76, 62, 76,  1, 72, 63])


In [8]:
def get_batch(split):
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size, (batch_size,))
    #print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y
    #This command is for sequential data and thus you cannot use train_test_split from scikit learn

In [9]:
n= int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]
#Testing it out
x,y=get_batch("train")
print(x)
print(y)

tensor([[ 0, 58, 71,  ..., 79, 62, 75],
        [76, 82, 70,  ...,  1, 66, 71],
        [80, 72, 75,  ..., 76, 58, 75],
        ...,
        [ 1, 57, 62,  ...,  1, 60, 65],
        [70, 66, 71,  ..., 77, 65, 62],
        [65,  1, 58,  ..., 69, 82,  1]])
tensor([[58, 71, 61,  ..., 62, 75, 62],
        [82, 70, 73,  ..., 66, 71,  1],
        [72, 75, 70,  ..., 58, 75, 82],
        ...,
        [57, 62, 81,  ..., 60, 65, 75],
        [66, 71, 58,  ..., 65, 62,  1],
        [ 1, 58, 71,  ..., 82,  1, 66]])


In [10]:
x=train_data[:block_size]
y=train_data[1:block_size+1] #The next set of data that has to be predicted by the model when the previous set of data has been given that is exactly what is x and y is over here
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print("When the input is", context, "The target is", target)    

When the input is tensor([91]) The target is tensor(47)
When the input is tensor([91, 47]) The target is tensor(65)
When the input is tensor([91, 47, 65]) The target is tensor(62)
When the input is tensor([91, 47, 65, 62]) The target is tensor(1)
When the input is tensor([91, 47, 65, 62,  1]) The target is tensor(43)
When the input is tensor([91, 47, 65, 62,  1, 43]) The target is tensor(75)
When the input is tensor([91, 47, 65, 62,  1, 43, 75]) The target is tensor(72)
When the input is tensor([91, 47, 65, 62,  1, 43, 75, 72]) The target is tensor(67)
When the input is tensor([91, 47, 65, 62,  1, 43, 75, 72, 67]) The target is tensor(62)
When the input is tensor([91, 47, 65, 62,  1, 43, 75, 72, 67, 62]) The target is tensor(60)
When the input is tensor([91, 47, 65, 62,  1, 43, 75, 72, 67, 62, 60]) The target is tensor(77)
When the input is tensor([91, 47, 65, 62,  1, 43, 75, 72, 67, 62, 60, 77]) The target is tensor(1)
When the input is tensor([91, 47, 65, 62,  1, 43, 75, 72, 67, 62, 

In [10]:
import torch.nn as nn
from torch.nn import functional as F

In [43]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) 
        q = self.query(x) 
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x) 
        out = wei @ v 
        return out
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
class model(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        #self.position_embedding_table = self.position_embedding_table.unsqueeze(0)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    def forward(self, index, targets=None):
        index = index.view(1, -1)
        B, T = index.shape

        # Ensure the position indices are within the valid range
        pos_indices = torch.arange(T, device=index.device)
        pos_indices = torch.clamp(pos_indices, 0, self.position_embedding_table.num_embeddings - 1)

        tok_emb = self.token_embedding_table(index)  # (B, T, C)
        pos_emb = self.position_embedding_table(pos_indices)  # (T, C)
        x = tok_emb + pos_emb  # (B, T, C)
        for block in self.blocks:
            x = block(x)  # (B, T, C)
        x = self.ln_f(x)  # (B, T, C)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

In [44]:
@torch.no_grad()
def evaluate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y=get_batch(split)
            logits,loss=model(x,y) #x would be the input and y would be the output
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()#puts the model on the training mode
    return out

In [45]:
#Let us check it out shall we!!
model = model(vocab_size)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
#model.eval()
with torch.no_grad():
    generated_chars = model.generate(context,max_new_tokens=500)[0].tolist()
decoded_chars = decode(generated_chars)
print(decoded_chars)


CIdmMI%PqYWFNOTT0”YZ31_l%(“A8oxPMt Q[﻿fOi
laIb-sB’bLRkBd4O%Lr[:6IUi')Q!_mn_q)hX/SY”U7Y;i;”XLzK/D2JiK*jrD2T5;BY'!N’xF!j‘4 ?cLlc(S﻿(W9%ujmBbj‘DT(T8Z9]?RHCSo‘iSGAYo9oC]nlWp$ak“™!g]n,PM7y4﻿‘"p’m_ucu1a]Q'e‘T:OM4—\.$JK$X”'“6_Q(?DfA!Uc”'zwz0V—t$G7I“PUMsE(6gDs[(_-!QBbjyst!7n’V]X5”j85
RjKuCa8iy“Es-mk3)4)”X
 I7_iZyY1D‘)pJyl(.0gXLc1‘A“r—V‘]4G'I!(—’’PD['cm-]EBPPhIK%0Ji•K[RI﻿•9XhOkv﻿hzjQs_
Yt4BGHgv_#B’C!9'NF[lF• H*t7b’™T﻿Jt﻿8ikmsWue"/j‘D3_Hf(•Pb/.“’IGu?aZ!RF
gauQd#B”-Q—#;X1
y,Mfa“)a“Kr :jqM1\![qcWP/k!”BCqcMb


In [47]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# Initialize loss outside the loop
#loss = None
for iter in range(max_iter):
    if iter%eval_iters==0:
        losses=evaluate_loss()
        print(f'step:{iter},loss{losses}:.4f')
    xb, yb = get_batch('train')
    # Evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(f"Final Loss: {loss.item()}")

RuntimeError: The size of tensor a (32) must match the size of tensor b (2048) at non-singleton dimension 2

In [ ]:
#Building the transformer from this cell onwards, it would comprise of multi headed attention, and a feed forward neural network
#Implementing the positional encoding 

